# HW Assignment 2

Implement both PG and an evolutionary algorithm to solve the Open AI Gym Lunar Lander problem, and then apply it to my area of choice, which is chess.

First, we need to do some setup

In [1]:
import torch
import numpy as np
import gym

# next few lines adapted from https://towardsdatascience.com/installing-pytorch-on-apple-m1-chip-with-gpu-acceleration-3351dc44d67c

print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Set the device
if torch.cuda.is_available():
    device = "gpu" # 🧮
elif torch.backends.mps.is_available():
    device = "mps" # 🧠
else:
    device = "cpu" # 🥺
    
print(f"Using device: {device}")

PyTorch version: 1.14.0.dev20221017
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Using device: mps


First, we need to write the code for our Policy Gradient function with a baseline (REINFORCE). I'm going to use PyTorch as my neural network library.

I'm going to start with a basic feed forward net for both the network that chooses the policy and the network that learns states' values.

First, the policy network for choosing actions

In [3]:
from torch import nn

class PolicyChoice(nn.Module):
    def __init__(self):
        super(PolicyChoice, self).__init__()
        self.flatten = nn.Flatten()
        self.policy = nn.Sequential(
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 4), # TODO: try reducing to one hidden layer if learning proves initially dificult
            nn.ReLU(),
            nn.Linear(4, 4),
            nn.Softmax() # to have a nice interpretation as probabilities of choosing actions
        )

    def forward(self, x):
        probs = self.policy(x)
        return probs

Now, the network for approximating state utililities.

Let's load the Lunar Lander environment now

In [2]:
env = gym.make(
    "LunarLander-v2",
    continuous = False,
    gravity = -10.0,
    enable_wind = False,
    wind_power = 15.0,
    turbulence_power = 1.5,
)

for episode in range(10000):
    action = policy(observation)  # User-defined policy function
    observation, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
      observation, info = env.reset()
env.close()